# OpenWeather DataFrame for ETL Project

We get our Auto Insurance Data, use the zip codes (or cities) in our dataset and create an API to pull weather data for those zip codes

#### Import dependencies

In [1]:
import matplotlib.pyplot as plt
import requests
import pandas as pd
from config import api_key
from pprint import pprint
import os
import pandas as pd
import csv

In [2]:
# # Import Auto Insurance data
auto_data_path = os.path.join("Resources", "Average_Auto_Insurance_Data_FINAL.csv")
auto_data_df = pd.read_csv(auto_data_path)

In [3]:
# # Set up a dataframe for the Auto insurance data
auto_data_df_v0 = auto_data_df .copy()
auto_data_df['Zip_Code']=auto_data_df_v0['Zip_Code'].apply(lambda x: '{0:0>5}'.format(x))
# auto_data_df.head(50)

In [4]:
x_df = auto_data_df['Zip_Code'].value_counts()
x_df.head(50)
x_df.describe()

count    175.000000
mean       1.125714
std        0.332478
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        2.000000
Name: Zip_Code, dtype: float64

In [5]:
# # Generate My lists
auto_data_list = auto_data_df.values.tolist()
zip_list = auto_data_df.Zip_Code.tolist() 
city_list1 = auto_data_df.City.tolist()

print(len(auto_data_list))
print(len(zip_list))
print(len(city_list1))

197
197
197


#### Save configuration and build query URL

In [6]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

# Build partial city query URL
query_url = f"{url}appid={api_key}&units={units}&q="
# print(query_url)

# Build partial zip code query URL
query2_url = f"{url}&units={units}&q="
# print(query2_url)

#### Loop through list of zip codes to request weather information for each

In [7]:
# Loop with a list of zipcodes

# set up lists to hold reponse info
lat_list = []
lon_list = []
hum_list = []
temp_list = []
maxTemp_list = []
minTemp_list = []
cloud_list = []
wind_list = []
city_list = []
weather_list = []
description_list =[]
# Loop through the list of cities and perform a request for data on each
for zips in zip_list:
    weather_url = f"{query2_url}{zips},US&appid={api_key}"
#     print(weather_url)
    response = requests.get(weather_url).json()
#     pprint(response)
    try:
        lon_list.append(response['coord']['lon'])
        lat_list.append(response['coord']['lat'])
    except:
        print(f"Oops, looks like we didn't get something (lat or long) from this zip code: {zips} gave us a problem. On to the next one")
        lon_list.append('NaN')
        lat_list.append('NaN')
    try:
        weather_list.append(response['weather'][0]['main'])
        description_list.append(response['weather'][0]['description'])
    except:
        print(f"Oops, looks like we didn't get something (weather or description) from this zip code: {zips} gave us a problem. On to the next one")
        weather_list.append('NaN')
        description_list.append('NaN')
    try:
        temp_list.append(response['main']['temp'])
        maxTemp_list.append(response['main']['temp_max'])
        minTemp_list.append(response['main']['temp_min'])
    except:
        print(f"Oops, looks like we didn't get something (temp, minTemp or maxtemp) from this zip code: {zips} gave us a problem. On to the next one")
        temp_list.append('NaN')
        maxTemp_list.append('NaN')
        minTemp_list.append('NaN')
    try:
        hum_list.append(response['main']['humidity'])
        cloud_list.append(response['clouds']['all'])
        wind_list.append(response['wind']['speed'])
        city_list.append(response['name'])
    except:
        print(f"Oops, looks like we didn't get something from this zip code: {zips} gave us a problem. On to the next one")
        hum_list.append('NaN')
        cloud_list.append('NaN')
        wind_list.append('NaN')
        city_list.append('NaN')

# Create lat lon pairs
latlongList = list(zip(lat_list, lon_list))
# latlongList

Oops, looks like we didn't get something (lat or long) from this zip code: 82725 gave us a problem. On to the next one
Oops, looks like we didn't get something (weather or description) from this zip code: 82725 gave us a problem. On to the next one
Oops, looks like we didn't get something (temp, minTemp or maxtemp) from this zip code: 82725 gave us a problem. On to the next one
Oops, looks like we didn't get something from this zip code: 82725 gave us a problem. On to the next one
Oops, looks like we didn't get something (lat or long) from this zip code: 99820 gave us a problem. On to the next one
Oops, looks like we didn't get something (weather or description) from this zip code: 99820 gave us a problem. On to the next one
Oops, looks like we didn't get something (temp, minTemp or maxtemp) from this zip code: 99820 gave us a problem. On to the next one
Oops, looks like we didn't get something from this zip code: 99820 gave us a problem. On to the next one
Oops, looks like we didn't g

#### Create a DataFrame from the zip code weather data

In [8]:
# Building df with zip code query
weather_dict = {"Zip_Code": zip_list, "Weather_Forecast": weather_list, "Weather_Description": description_list, "Max_Temperature": maxTemp_list, "Min_Temperature": minTemp_list, "Humidity": hum_list}
weather_df = pd.DataFrame(weather_dict)
weather_df = weather_df[~weather_df.Max_Temperature.str.contains("NaN", na=False)]
weather_df = weather_df.reset_index()
weather_df

,index,Zip_Code,Weather_Forecast,Weather_Description,Max_Temperature,Min_Temperature,Humidity
0,0,48226,Clouds,overcast clouds,41,37.99,93
1,1,11212,Clouds,overcast clouds,57.2,55,67
2,2,70117,Clouds,overcast clouds,80.01,77,100
3,3,89101,Clear,clear sky,66.2,57,13
4,4,91605,Clear,clear sky,68,59,15
...,...,...,...,...,...,...,...
187,192,30314,Clouds,broken clouds,69.8,66.2,100
188,193,70117,Clouds,overcast clouds,80.01,77,100
189,194,20002,Clouds,overcast clouds,61,57.99,87
190,195,07102,Clouds,overcast clouds,57.2,53.01,62


#### Merge weather data with Auto insurance data

In [10]:
# Combine the data into a single dataset
Auto_data_weather_df = pd.merge(auto_data_df, weather_df, how='left', on='Zip_Code')
# Auto_data_weather_v1a_df = Auto_data_weather_v1_df.rename(columns={'City_y':'City'})
Auto_data_weather_df = Auto_data_weather_df.dropna(how='any')
# Auto_data_weather_v1_df.reset_index()
Auto_data_weather_df

,Index,State,Zip_Code,City,Average_Auto_Insurance_Rate,index,Weather_Forecast,Weather_Description,Max_Temperature,Min_Temperature,Humidity
0,0,Michigan,48226,Detroit,6329.00,0.0,Clouds,overcast clouds,41,37.99,93
1,0,Michigan,48226,Detroit,6329.00,63.0,Clouds,overcast clouds,41,37.99,93
2,1,New York,11212,New York,5703.00,1.0,Clouds,overcast clouds,57.2,55,67
3,1,New York,11212,New York,5703.00,158.0,Clouds,overcast clouds,57.2,55,67
4,2,Louisiana,70117,New Orleans,4601.00,2.0,Clouds,overcast clouds,80.01,77,100
...,...,...,...,...,...,...,...,...,...,...,...
236,195,Louisiana,70117,New Orleans,9303.50,2.0,Clouds,overcast clouds,80.01,77,100
237,195,Louisiana,70117,New Orleans,9303.50,193.0,Clouds,overcast clouds,80.01,77,100
238,196,District Of Columbia,20002,Washington,4440.79,194.0,Clouds,overcast clouds,61,57.99,87
239,197,New Jersey,07102,Newark,8512.59,195.0,Clouds,overcast clouds,57.2,53.01,62


In [ ]:
# # Combine the data into a single dataset
# Auto_df = pd.merge(auto_data_df, weather_df, how='left', on='Zip_Code')
# # Auto_data_weather_v1a_df = Auto_data_weather_v1_df.rename(columns={'City_y':'City'})
# Auto_data_weather_df = Auto_data_weather_v2_df.dropna(how='any')
# # Auto_data_weather_v1_df.reset_index()
# Auto_df

#### Outputing data

In [12]:
# Output weather dataset
output_file = os.path.join("Resources", "Weather_Data_Final.csv")
weather_df.to_csv(output_file, index = False)

# Output merged dataset
output_file = os.path.join("Resources", "Average_Auto_Insurance_Weather_Data_Final.csv")
Auto_data_weather_df.to_csv(output_file, index = True)